# Predictions for car battery life

## Problem overview:

This scenario focuses on predictions for car batteries reaching their rated number of cycles. The goal here is to avoid a sudden failure of the car battery and schedule it to be replaced before it reaches its rated number of cycles. The reduces significantly the risk of collateral failures occuring as well. From a business point of view, the target is optimize car battery replacement schedules to minimize overall maintenance costs. 

The connected cars report on a daily basis information about the number of trips, duration of those trips and number of battery cycles used. Since reporting is performed on a daily basis, the battery age measured in days is also available as an input. The accumulated lifetime cycles used measure is also part of the dataset. As you can easily guess, the input data comes in the form of timeseries with the **Lifetime_Cycles_Used** field being the subject of prediction. Given the current timeseries for each battery, we are interested in forecasting the value of this field for the next 30 days to see whether the rated number of cycles will be exceeeded or not. 

This scenario details the development of a machine learning time series forecasting model. The model is trained on a dataset containing battery telemetry information for 1539 days (between 1/1/2013 and 3/19/2017) for one car battery.

## Solution overview:

We will use the [Automated Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train) (autoML) capabilities of the [Azure Machine Learning service](https://docs.microsoft.com/en-us/azure/machine-learning/service/overview-what-is-azure-ml) to quickly train a model that can forecast the evolution of the **Lifetime_Cycles_Used** metric. We will model our problem as a **Forecasting** problem where the goal of the trained model is to forecast the future evolution of a numerical indicator over a number of cycles (30 days in our case). The automML capabilities enable us to evaluate different algorithms and hyperparameters to get the best trained model for the problem with minimum effort. The approach used in this example cand be extended to various use cases that revolve around the need to predict the time series-based evolution of a numerical value.


This notebook is organized into the following sections:

1. Basic setup

2. Data prep

3. Model training

4. Explore the results and evaluate the best model

5. Model Explainability: which features matter for the predictions?

## Section 1. Basic setup



Install the ```tqdm``` module which is required by forecasting model explainability.

In [ ]:
!pip install --upgrade tqdm

Import the necessary namespaces and classes.

In [ ]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

from matplotlib import pyplot as plt

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.automlexplainer import explain_model

Before starting this step, you need to create an Azure Machine Learning service workspace ([instructions](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace)).

Let's get started by creating an experiment in your Azure Machine Learning workspace. An experiment is a named object in a workspace, which is used to do model training.

In [ ]:
subscription_id = "<subscription id goes here>"
resource_group = "<resource group goes here>"
workspace_name = "<workspace name goes here>"

In [ ]:
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

In [ ]:
# choose a name for the run history container in the workspace
experiment_name = 'CarBatteryLife'

# project folder
project_folder = './sample_projects/automl-carbatterylifeforecast'

experiment=Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

## Section 2. Data prep

```daily-battery-time-series.csv``` contains the data about the time-based evolution of the car battery parameters. It contains information for one battery during 1539 daily cycles (between 2013-01-01 and 2017-03-19).

In [ ]:
data = pd.read_csv("https://quickstartsws9073123377.blob.core.windows.net/amlnotebooktutorials/customer-lifetime-value/daily-battery-time-series.csv", parse_dates=['Date'])

## 2.1 Inspect data

Display the first few rows of the data and view some plots to help you understand the dynamics within the dataset.

In [ ]:
data.head(20)

Let's take a look at some of the dataset properties.

In [ ]:
fig = plt.figure(figsize=(20,6))

ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.plot(data['Daily_Trip_Duration'])
ax2.plot(data['Lifetime_Cycles_Used'])
fig.show()

Since the dataset contains informatio for a single battery, we reduce the problem to forecasting the evolution of this single battery.

In [ ]:
data = data[['Date', 'Battery_Age_Days', 'Number_Of_Trips', 'Daily_Trip_Duration', 'Daily_Cycles_Used', 'Lifetime_Cycles_Used']]
data.describe()

## 2.2 Split the data into train and test sets

When working with timeseries, the approach used for split is to use the first X% occurences from the time series as training data and the remaining (100-X)% occurences as test data. Once the forecasting model is trained, it is used to make predictions for the occurrences from the test data. The differences between the predicted values and the actual values from the test data are used to measure the performance of the model.

In [ ]:
time_column_name = 'Date'
target_column_name = 'Lifetime_Cycles_Used'

X_train = data[data[time_column_name] < '2016-04-15']
X_test = data[data[time_column_name] >= '2016-04-15']
y_train = X_train.pop(target_column_name).values
y_test = X_test.pop(target_column_name).values

## Section 3. Model training

In this section you will configure the [automated ML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml) feature of the Azure Machine Learning service. Using the configuration you will then run an automated ML experiment which explores various algorithms and hyperparameter values to generate machine learning models. Finally, the best model will be selected. The training jobs run on local compute resources (provided and managed by Azure Notebooks).

The significant advantage of automated ML is the acceleration of a data scientist's work (as it does a significant portion of the exploration work). Besides that, automated ML exposes rich data resulting from experiment runs which enables control, transparency, and, most importantly, visibility on what is happening behind the scenes.

The configuration data required by automated ML contains information about the experiment itself as well as the training data used to train the models. Below is an example of the most important components of the configuration data:

Property | Description
--- | ---
task | regression
primary_metric | Metric that you want to optimize.<br>Forecasting supports the following primary metrics:<br>spearman_correlation<br>normalized_root_mean_squared_error<br>r2_score<br>normalized_mean_absolute_error
iterations | Number of iterations. In each individual iteration, automated ML trains one pipeline (algorithm and hyperparameters) on the given data.
iteration_timeout_minutes | The maximum number of minutes for each individual iteration
X | Training data in the form [n_samples, n_features]
y | Target values in the form [n_samples]
n_cross_validations | Number of [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) splits.
path | Relative path to the project folder. Automated ML stores configuration files for the experiment under this folder. You can specify a new empty folder.


### 3.1 Automated ML configuration

Automated ML provides several options to configure the experiment runs, giving you flexibility and control. For example, the primary_metric setting specifies which metrich should automated ML use to optimize the machine learning model being built. There are multiple primary metrics available, and in our problem we will use NRMSE (Normalized Root Mean Squared Error).

Notice that we've set the task to forecasting and we are also specifying the training data set (X_train and y_train). We need to do this because training is performed locally. When training in performed remotely (e.g. on AML compute resources) you will need to provide a script that contains code to get the data instead of the data itself.

Also, we are indicating the time column name and the maximum time horizon for the forecast (30 cycles which amount to 30 days in our case).

In [ ]:
automl_settings = {
    "time_column_name": time_column_name, 
    "max_horizon": 30
}

automl_config = AutoMLConfig(task = 'forecasting',
                             debug_log = 'automl_carbatterylife_errors.log',
                             primary_metric= 'normalized_root_mean_squared_error',
                             iterations = 5,
                             iteration_timeout_minutes = 5,
                             X = X_train,
                             y = y_train,
                             n_cross_validations = 3,
                             path=project_folder,
                             verbosity = logging.INFO,
                            **automl_settings)

### 3.2 Train your models on local compute

When you call the submit method on the experiment object and pass the AutoMLConfig object, automated ML will general a number of machine learning models equal to **iterations** (5 in our case). Depending on the input data and the number of iterations the time required to complete the traning can range from a few minutes to hours (or even mode). Once execution starts, you will see status messages being print out to the console.

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

### 3.3 Monitor training

To see the detailed results of the run, you can use a special widget (see the cell below).

**Note**: When you want to start the run and continue to execute your code you need to specify ```show_output=False``` when calling ```experiment.submit()```.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

## Section 4. Explore the results and test the best model

## 4.1 Retrieve all child runs

Each individual model is trained in the context of a child run having **local_run** as its parent. You can get the list of all child runs and their logged metrics.

In [ ]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

## 4.2 Retrieve the best fitted model

The **get_output** method enables you to retrieve the best child run and the associated fitted model.

In [ ]:
best_run, fitted_model = local_run.get_output()
fitted_model.steps

## 4.3 Test the best fitted model

We will now use the best model trained by automated ML to make predictions on the test data. Finally, the RMSE (Root Mean Squared Error) value will be calculated for the test data.

In [ ]:
y_pred_test = fitted_model.predict(X_test)
y_diff = np.abs(y_test - y_pred_test)

rmse_test = ((y_test - y_pred_test) ** 2).mean() ** .5
print('RMSE for test data is {}'.format(rmse_test))

To get a visual representation of the result, let's display a couple of charts.

In [ ]:
fig = plt.figure(figsize=(30,15))

ax1 = fig.add_subplot(211)
ax1.plot(X_test['Date'], y_test, label='Actual')
ax1.plot(X_test['Date'], y_pred_test, label="Predicted")
ax1.set_title('Actual battery life vs predicted for test data ')
ax1.legend()
plt.xticks(rotation=90)

ax2 = fig.add_subplot(212)
ax2.plot(X_test['Date'], y_diff)
ax2.set_title('Difference between actual and predicted for test data ')
plt.xticks(rotation=90)

fig.show()

## Section 5. Model Explainability: Which features matter for the predictions?

Now that we have our trained model, we'd like to understand aspects related to its inner workings. One of the most important questions is "Which feature matters the most in calculating the predictions?".

To get the answer to the question, let's retrieve the explanation for our model.

In [ ]:
shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = explain_model(fitted_model, X_train, X_test, features=X_train.columns, best_run=best_run, y_train=y_train)

```overall_imp``` contains an ordered list of the features according to their importance. Notice that the list of features includes both features from the original dataset as well as features engineered by automated ML during the training process.

In [ ]:
overall_imp